# Azure AI Agent with Function Tools Example

This notebook demonstrates function tool integration with Azure AI Agents, showing both agent-level and query-level tool configuration patterns.

## Features Covered:
- Defining function tools for agents
- Agent-level tool configuration
- Run-method tool configuration
- Mixed tool usage patterns
- Multiple tool coordination

## Prerequisites

- Azure CLI authenticated (`az login --use-device-code`)
- Azure AI Foundry project with deployed models
- Environment variables configured

## Import Libraries

In [ ]:
import os
from pathlib import Path
import asyncio
from datetime import datetime, timezone
from random import randint
from typing import Annotated

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity import AzureCliCredential
from pydantic import Field
from dotenv import load_dotenv

# Load environment variables from base directory
load_dotenv('../../../.env')

# Get connection string from environment variables
conn_string = os.getenv("PROJECT_CONNECTION_STRING")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME", "gpt-4o")

## Define Function Tools

In [ ]:
# Validate configuration
if not conn_string:
    raise ValueError("PROJECT_CONNECTION_STRING not found in environment variables")
if not model_deployment:
    raise ValueError("MODEL_DEPLOYMENT_NAME not found in environment variables")

print(f"Configuration loaded successfully")
print(f"Model deployment: {model_deployment}")

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

def get_time() -> str:
    """Get the current UTC time."""
    current_time = datetime.now(timezone.utc)
    return f"The current UTC time is {current_time.strftime('%Y-%m-%d %H:%M:%S')}."

## Tools at Agent Level

In [ ]:
async def tools_on_agent_level():
    """Demonstrate tools defined at agent creation."""
    print("=== Tools Defined at Agent Level ===")

    credential = AzureCliCredential()
    
    # Agent created with predefined tools
    async with ChatAgent(
        chat_client=AzureAIAgentClient(
            project_endpoint=conn_string,
            model_deployment_name=model_deployment,
            async_credential=credential
        ),
        instructions="You are a helpful assistant that can provide weather and time information.",
        tools=[get_weather, get_time],  # Tools defined at agent creation
    ) as agent:
        # First query - agent can use weather tool
        query1 = "What's the weather like in New York?"
        print(f"User: {query1}")
        result1 = await agent.run(query1)
        print(f"Agent: {result1.text}\n")

        # Second query - agent can use time tool
        query2 = "What's the current UTC time?"
        print(f"User: {query2}")
        result2 = await agent.run(query2)
        print(f"Agent: {result2.text}\n")

        # Third query - agent can use both tools if needed
        query3 = "What's the weather in London and what's the current UTC time?"
        print(f"User: {query3}")
        result3 = await agent.run(query3)
        print(f"Agent: {result3.text}\n")

# Run the agent-level tools example
await tools_on_agent_level()

## Tools at Run Level

In [ ]:
async def tools_on_run_level():
    """Demonstrate tools passed to the run method."""
    print("=== Tools Passed to Run Method ===")

    credential = AzureCliCredential()
    
    # Agent created without tools
    async with ChatAgent(
        chat_client=AzureAIAgentClient(
            project_endpoint=conn_string,
            model_deployment_name=model_deployment,
            async_credential=credential
        ),
        instructions="You are a helpful assistant.",
        # No tools defined here
    ) as agent:
        # First query with weather tool
        query1 = "What's the weather like in Seattle?"
        print(f"User: {query1}")
        result1 = await agent.run(query1, tools=[get_weather])
        print(f"Agent: {result1.text}\n")

        # Second query with time tool
        query2 = "What's the current UTC time?"
        print(f"User: {query2}")
        result2 = await agent.run(query2, tools=[get_time])
        print(f"Agent: {result2.text}\n")

        # Third query with multiple tools
        query3 = "What's the weather in Chicago and what's the current UTC time?"
        print(f"User: {query3}")
        result3 = await agent.run(query3, tools=[get_weather, get_time])
        print(f"Agent: {result3.text}\n")

# Run the run-level tools example
await tools_on_run_level()

## Mixed Tools Example

In [ ]:
async def mixed_tools_example() -> None:
    """Example showing both agent-level tools and run-method tools."""
    print("=== Mixed Tools Example (Agent + Run Method) ===")

    credential = AzureCliCredential()
    
    # Agent created with some base tools
    async with ChatAgent(
        chat_client=AzureAIAgentClient(
            project_endpoint=conn_string,
            model_deployment_name=model_deployment,
            async_credential=credential
        ),
        instructions="You are a comprehensive assistant that can help with various information requests.",
        tools=[get_weather],  # Base tool available for all queries
    ) as agent:
        # Query using both agent tool and additional run-method tools
        query = "What's the weather in Denver and what's the current UTC time?"
        print(f"User: {query}")

        # Agent has access to get_weather (from creation) + additional tools from run method
        result = await agent.run(
            query,
            tools=[get_time],  # Additional tools for this specific query
        )
        print(f"Agent: {result.text}\n")

# Run the mixed tools example
await mixed_tools_example()

## Advanced Tools

In [ ]:
def calculate(
    expression: Annotated[str, Field(description="A mathematical expression to evaluate (e.g., '2 + 3 * 4')")],
) -> str:
    """Calculate the result of a mathematical expression."""
    try:
        # Simple eval for basic math (in production, use a safer math parser)
        result = eval(expression)
        return f"The result of '{expression}' is {result}"
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"

def get_date_info(
    timezone_name: Annotated[str, Field(description="Timezone name (e.g., 'US/Eastern', 'Europe/London')")]
) -> str:
    """Get date and time information for a specific timezone."""
    try:
        from zoneinfo import ZoneInfo
        tz = ZoneInfo(timezone_name)
        current_time = datetime.now(tz)
        return f"Current time in {timezone_name}: {current_time.strftime('%Y-%m-%d %H:%M:%S %Z')}"
    except Exception:
        # Fallback to UTC if timezone is not available
        utc_time = datetime.now(timezone.utc)
        return f"UTC time (timezone '{timezone_name}' not available): {utc_time.strftime('%Y-%m-%d %H:%M:%S')}"

def convert_temperature(
    temperature: Annotated[float, Field(description="Temperature value to convert")],
    from_unit: Annotated[str, Field(description="Source unit: 'C', 'F', or 'K'")],
    to_unit: Annotated[str, Field(description="Target unit: 'C', 'F', or 'K'")]
) -> str:
    """Convert temperature between Celsius, Fahrenheit, and Kelvin."""
    try:
        # Convert to Celsius first
        if from_unit.upper() == 'F':
            celsius = (temperature - 32) * 5/9
        elif from_unit.upper() == 'K':
            celsius = temperature - 273.15
        else:  # Assume Celsius
            celsius = temperature
        
        # Convert from Celsius to target
        if to_unit.upper() == 'F':
            result = celsius * 9/5 + 32
        elif to_unit.upper() == 'K':
            result = celsius + 273.15
        else:  # Celsius
            result = celsius
        
        return f"{temperature}°{from_unit.upper()} = {result:.2f}°{to_unit.upper()}"
    except Exception as e:
        return f"Error converting temperature: {str(e)}"

## Comprehensive Tools Example

In [ ]:
async def comprehensive_tools_example():
    """Example with multiple sophisticated tools."""
    print("=== Comprehensive Tools Example ===")
    
    credential = AzureCliCredential()
    
    # Create an agent with multiple tools
    async with ChatAgent(
        chat_client=AzureAIAgentClient(
            project_endpoint=conn_string,
            model_deployment_name=model_deployment,
            async_credential=credential
        ),
        instructions=(
            "You are a multi-purpose assistant that can help with weather, time, "
            "calculations, and temperature conversions. Always use the appropriate "
            "tools to provide accurate information."
        ),
        tools=[
            get_weather,
            get_time,
            calculate,
            get_date_info,
            convert_temperature
        ],
    ) as agent:
        # Test various tool combinations
        queries = [
            "What's the weather in Paris?",
            "Calculate 15 * 23 + 47",
            "What time is it in Europe/London timezone?",
            "Convert 25 degrees Celsius to Fahrenheit",
            "What's the weather in Tokyo and what time is it there in Asia/Tokyo timezone?",
            "If it's 68°F outside, what's that in Celsius? Also calculate 68 * 1.5"
        ]
        
        for i, query in enumerate(queries, 1):
            print(f"\n--- Query {i} ---")
            print(f"User: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text}")

# Run the comprehensive tools example
await comprehensive_tools_example()

## Tool Selection Strategy

In [ ]:
async def tool_selection_example():
    """Example demonstrating intelligent tool selection."""
    print("=== Tool Selection Strategy Example ===")
    
    credential = AzureCliCredential()
    
    async with ChatAgent(
        chat_client=AzureAIAgentClient(
            project_endpoint=conn_string,
            model_deployment_name=model_deployment,
            async_credential=credential
        ),
        instructions=(
            "You are an intelligent assistant. Analyze each query and use the most "
            "appropriate tools. Explain which tools you're using and why."
        ),
        tools=[get_weather, get_time, calculate, convert_temperature],
    ) as agent:
        # Test queries that might need different tool combinations
        test_queries = [
            "I need just the current UTC time",
            "What's 2+2?",
            "Is it hot in Miami? What's 85°F in Celsius?",
            "Give me a weather report for London",
            "Help me plan: what's the weather in New York and what time is it there?"
        ]
        
        for query in test_queries:
            print(f"\nUser: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text}")

# Run the tool selection example
await tool_selection_example()